In [3]:
import pandas as pd
import numpy as np
from itertools import product

In [2]:
df = pd.read_excel(r'C:\Users\Lenovo\Desktop\kt1_electre.xlsx')
df

,Unnamed: 0,k1,k2,k3
0,A,180,70,10
1,B,170,40,15
2,C,160,55,20
3,D,150,50,25
4,weights,3,2,1
5,scale,100,50,45


In [15]:
def electre(df_, criteria_,method='C'):
    """
    :param df_: input DataFrame
    :param criteria: list of criterias where -1 belongs to min and 1 to max
    :param method: C or d
    """
    rows = df_.iloc[:,0].tolist()
    names = rows[:-2]
    weights = df_.iloc[rows.index('weights'),1:]
    scales = df_.iloc[rows.index('scale'),1:]
    
    data_ = df_.to_numpy().copy()
    combs = list(product(list(range(len(names))), repeat=2))
    w = sum(weights)
    criteria = np.array(criteria_)
    res = np.zeros((int(np.sqrt(len(combs))), int(np.sqrt(len(combs)))))
    if method=='C':
        for c1, c2 in combs:
            crit_1 =  data_[c1, 1:] * criteria
            crit_2 =  data_[c2, 1:] * criteria
            mask = crit_1 >= crit_2
            res[c1, c2] = (mask @ weights)/w
    elif method=='d':
        for c1, c2 in combs:
            crit_1 =  data_[c1, 1:] * criteria
            crit_2 =  data_[c2, 1:] * criteria
            mask = crit_1 >= crit_2
            res[c1, c2] = np.max(((1-mask)*crit_2 - (1-mask)*crit_1)/scales)
    else:
        raise Exception('Choose method "C" or "d"')

    return pd.DataFrame(res, index=names, columns=names)

In [17]:
electre(df,[-1,-1,-1],method='C')

,A,B,C,D
A,1.000000,0.166667,0.166667,0.166667
B,0.833333,1.000000,0.500000,0.500000
C,0.833333,0.500000,1.000000,0.166667
D,0.833333,0.500000,0.833333,1.000000


In [18]:
electre(df,[-1,-1,-1],method='d')

,A,B,C,D
A,0.000000,0.600000,0.300000,0.4
B,0.111111,0.000000,0.100000,0.2
C,0.222222,0.300000,0.000000,0.1
D,0.333333,0.222222,0.111111,0.0
